In [ ]:
# default_exp core

# SGNN

> Implementation of Self-Governing Neural Networks for speech act classification

Implementation of the [SGNN paper](https://www.aclweb.org/anthology/D19-1402.pdf) for speech act classification.
This repository is inspired by Guillaume Chevalier's [implementation](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer), as well as his [discussion](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer/issues/1) with [Sava Kalbachou](https://github.com/thinline).
This version implements some things differently from Guillaume's code, and extends beyond the projection layer all the way to a fully trainable network.

The network is trained to classify the [SwDA corpus](https://web.stanford.edu/~jurafsky/ws97/) utterances according to their speech act. The corpus was was pre-processed using Cristopher Pott's [project](https://github.com/cgpotts/swda/) related to it; the pre-processed data is included in [`data/swda-acttags-and-text.csv`](data/swda-acttags-and-text.csv) for repeateability.

In [ ]:
#hide
from nbdev.showdoc import *

We first import the data from file

In [ ]:
# export
import pandas as pd

data_filepath = 'data/swda-acttags-and-text.csv'
data = pd.read_csv(data_filepath)

In [ ]:
# See the data
data.describe

Divide data in test and train sets

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

data = data.dropna(axis=0)  # Drop rows with NA values
y = data.DamslActTag
X = data.Text
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

See the data division

In [ ]:
print(X_train.head(),"\n", y_train.head())

## Let's build SGNN the pipeline for processing the data

In [ ]:
#export

# CountVectorizer Parameters
char_ngram_range = (1, 4)

char_term_frequency_params = {
    'char_term_frequency__analyzer': 'char',
    'char_term_frequency__lowercase': True,
    'char_term_frequency__ngram_range': char_ngram_range,
    'char_term_frequency__strip_accents': None,
    'char_term_frequency__min_df': 2,
    'char_term_frequency__max_df': 0.99,
    'char_term_frequency__max_features': int(1e7),
}

In [ ]:
#export 
import scipy.sparse as sp

T = 80
d = 14
# T=80 projections for each of dimension d=14: 80 * 14 = 1120-dimensionnal word projections.
hashing_feature_union_params = {
    **{'union__sparse_random_projection_hasher_{}__n_components'.format(t): d
       for t in range(T)
    },
    **{'union__sparse_random_projection_hasher_{}__dense_output'.format(t): False  # only AFTER hashing.
       for t in range(T)
    }
}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.random_projection import SparseRandomProjection
from sklearn.neural_network import MLPClassifier

params = dict()
params.update(char_term_frequency_params)
params.update(hashing_feature_union_params)

preprocessor = Pipeline([
    ("char_term_frequency", CountVectorizer()),
    ('union', FeatureUnion([
        ('sparse_random_projection_hasher_{}'.format(t), SparseRandomProjection())
        for t in range(T)
    ])),
])
preprocessor.set_params(**params)

In [ ]:
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('neuralnet', MLPClassifier(
        hidden_layer_sizes=(256, 256), activation='logistic', solver='sgd', 
        batch_size=100, learning_rate='adaptive', learning_rate_init=0.025,
        verbose=True
        )
    )
])

pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)